### Datasets
http://www.manythings.org/anki/  (Download and unzip fra-eng.zip file)

In [1]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [3]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Using TensorFlow backend.


In [5]:
lines= pd.read_table('fra.txt', names=['eng', 'fra'],index_col=False,nrows=50000)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [6]:
lines.shape

(50000, 2)

In [7]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.fra=lines.fra.apply(lambda x: x.lower())

In [8]:
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.fra=lines.fra.apply(lambda x: re.sub("'", '', x))

In [9]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.fra=lines.fra.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [10]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
#lines.mar = lines.mar.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

In [11]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.fra=lines.fra.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.fra=lines.fra.apply(lambda x: re.sub(" +", " ", x))

In [12]:
# Add start and end tokens to target sequences
lines.fra = lines.fra.apply(lambda x : 'START_ '+ x + ' _END')

In [13]:
lines.sample(10)

eng                                          fra
49177  im a japanese teacher   START_ je suis enseignant en japonais _END
38122    theyre all tourists       START_ ce sont tous des touristes _END
36626    im very tired today  START_ je suis très fatigué aujourdhui _END
4001             im stubborn                  START_ je suis obstiné _END
33059    whose phone is that           START_ à qui est ce téléphone _END
8724           its your turn                  START_ cest votre tour _END
32666     well never make it        START_ on ne va jamais y arriver _END
17725        youre lying now         START_ tu es en train de mentir _END
19718      i will go on foot                     START_ jirai à pied _END
17368       which is correct               START_ lequel est correct _END

In [14]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of French 
all_french_words=set()
for fra in lines.fra:
    for word in fra.split():
        if word not in all_french_words:
            all_french_words.add(word)

In [15]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

7

In [16]:
# Max Length of target sequence
lenght_list=[]
for l in lines.fra:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

16

In [17]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_french_words))
num_encoder_tokens = len(all_eng_words)+1
num_decoder_tokens = len(all_french_words)+1
num_encoder_tokens, num_decoder_tokens

(5952, 12577)

In [18]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

12578

In [19]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [20]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [21]:
lines = shuffle(lines)
lines.head(10)

eng                                    fra
21411    tom could do that      START_ tom pourrait faire ça _END
9203        they were busy        START_ ils étaient occupés _END
7759         hows your job      START_ comment va ton boulot _END
25124   is there any sugar            START_ y atil du sucre _END
34254  dont try to fool me    START_ nessaie pas de me duper _END
23712   hedgehogs are cute  START_ les hérissons cest mignon _END
25493   life is so strange      START_ la vie est si étrange _END
30555   ive made my choice         START_ jai fait mon choix _END
9343        tom looks sick   START_ tom a lair plutôt malade _END
21654    we all felt happy   START_ nous étions tous heureux _END

In [22]:
# Train - Test Split
X, y = lines.eng, lines.fra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((45000,), (5000,))

#### Save the train and test dataframes for reproducing the results later, as they are shuffled.

In [23]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [24]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder - Decoder Model Architecture

In [25]:
latent_dim = 50

In [26]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [27]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [54]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [55]:
# from IPython.display import Image
# Image(retina=True, filename='train_model.png')

In [56]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50

print(train_samples//batch_size)
print(val_samples//batch_size)

351
39


In [57]:
model.fit(generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/50
351/351 [==============================] - 413s 1s/step - loss: 1.9222 - acc: 0.2009 - val_loss: 1.7074 - val_acc: 0.2359
Epoch 2/50
351/351 [==============================] - 405s 1s/step - loss: 1.6123 - acc: 0.2624 - val_loss: 1.5582 - val_acc: 0.2866
Epoch 3/50
351/351 [==============================] - 405s 1s/step - loss: 1.4797 - acc: 0.3118 - val_loss: 1.4379 - val_acc: 0.3423
Epoch 4/50
351/351 [==============================] - 407s 1s/step - loss: 1.3708 - acc: 0.3589 - val_loss: 1.3538 - val_acc: 0.3749
Epoch 5/50
351/351 [==============================] - 406s 1s/step - loss: 1.2922 - acc: 0.3887 - val_loss: 1.2921 - val_acc: 0.3984
Epoch 6/50
351/351 [==============================] - 403s 1s/step - loss: 1.2304 - acc: 0.4143 - val_loss: 1.2436 - val_acc: 0.4204
Epoch 7/50
351/351 [==============================] - 402s 1s/step - loss: 1.1795 - acc: 0.4342 - val_loss: 1.2014 - val_acc: 0.4393
Epoch 8/50
351/351 [==============================] - 405s 1s/step - 

### Always remember to save the weights

In [58]:
model.save_weights('nmt_weights.h5')

### Load the weights, if you close the application

In [29]:
model.load_weights('nmt_weights_67_percent_accuracy.h5')

### Inference Setup

In [30]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [1]:
# Image(retina=True, filename='encoder_model.png')

In [ ]:
# Image(retina=True, filename='decoder_model.png')

### Decode sample sequeces

In [31]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

### Evaluation on Train Dataset

In [103]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [104]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual French Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted French Translation:', decoded_sentence[:-4])

Input English sentence: here is your bag
Actual French Translation:  ton sac est ici 
Predicted French Translation:  voici votre sac 


In [107]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual French Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted French Translation:', decoded_sentence[:-4])

Input English sentence: youre very funny
Actual French Translation:  vous êtes très marrant 
Predicted French Translation:  vous êtes très drôle 


In [38]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual French Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted French Translation:', decoded_sentence[:-4])

Input English sentence: how long did you stay
Actual French Translation:  combien de temps estu resté 
Predicted French Translation:  combien de temps êtesvous 


### Evaluation on Validation Dataset

In [108]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [79]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual French Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted French Translation:', decoded_sentence[:-4])

Input English sentence: let go of my hand
Actual French Translation:  lâchezmoi la main 
Predicted French Translation:  laissemoi men aller 


In [80]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual French Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted French Translation:', decoded_sentence[:-4])

Input English sentence: i dont want to know
Actual French Translation:  je ne veux pas le savoir 
Predicted French Translation:  je ne veux pas le savoir 


In [81]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual French Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted French Translation:', decoded_sentence[:-4])

Input English sentence: let me go
Actual French Translation:  lâchemoi 
Predicted French Translation:  laissezmoi partir 
